In [1]:
import utils.losses as loss
import datasets.datasets as dtset

In [2]:
Data = dtset.OpenML_Dataset('iris')

c:\Users\danibacaicoa\vscode_projects\Learning_from_weak_labels\datasets\datasets.py:86: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  data = openml.datasets.get_dataset(openml_ids[self.dataset])


In [3]:
train_X,train_y,test_X,test_y =  Data.get_data()
print(train_X.shape)

torch.Size([105, 4])


In [4]:
print(train_X[:5,:])
print(train_y[:5,:])

tensor([[5.6000, 2.8000, 4.9000, 2.0000],
        [5.6000, 2.9000, 3.6000, 1.3000],
        [6.7000, 2.5000, 5.8000, 1.8000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [4.4000, 2.9000, 1.4000, 0.2000]])
tensor([[0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]])


In [5]:
import torch
import torch.nn as nn

# Define the MLP class
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    # Define layers
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.softmax = nn.Softmax(dim=1)  # For classification tasks

  def forward(self, x):
    # Forward pass through the layers
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

model = MLP(4, 2, 3)

prediction = model(train_X)
print(prediction)


tensor([[-3.2267,  1.0525, -3.7203],
        [-2.6616,  0.9027, -3.6360],
        [-3.6354,  1.2185, -4.0475],
        [-3.4079,  1.1829, -4.1272],
        [-1.8166,  0.5079, -2.7220],
        [-2.3070,  0.7371, -3.2525],
        [-1.7406,  0.4724, -2.6398],
        [-2.3115,  0.7392, -3.2573],
        [-3.9081,  1.3274, -4.2570],
        [-1.9479,  0.5693, -2.8641],
        [-1.9168,  0.5547, -2.8304],
        [-2.2038,  0.6889, -3.1409],
        [-3.4256,  1.1274, -3.8524],
        [-2.1465,  0.6621, -3.0789],
        [-3.5956,  1.2502, -4.2362],
        [-4.3203,  1.5347, -4.7703],
        [-3.7764,  1.2827, -4.1919],
        [-2.3136,  0.7401, -3.2596],
        [-2.0953,  0.6382, -3.0235],
        [-2.2802,  0.7245, -3.2235],
        [-3.9893,  1.3681, -4.3573],
        [-2.5615,  0.8560, -3.5278],
        [-2.6515,  0.8752, -3.5197],
        [-1.9159,  0.5543, -2.8294],
        [-2.8148,  0.9531, -3.7037],
        [-3.0096,  1.0555, -3.9668],
        [-3.5865,  1.2296, -4.1509],
 

In [6]:
train_dataloader,test_dataloader =  Data.get_dataloader()

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = loss.GumbelLoss()

In [8]:
# Training loop (with improvements):
for epoch in range(101):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # Get the inputs; forward pass:
        inputs, labels = data
        outputs = model(inputs)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(running_loss)
    
print('Finished training!')

NameError: name 'gumbel_softmax' is not defined

In [ ]:
for images, labels in test_dataloader:
    outputs = model(images)
    outputs
    print(torch.max(outputs.data, 1))

torch.return_types.max(
values=tensor([2.2311, 1.2917, 1.1168, 3.3529, 2.8421, 1.2917, 2.2608, 1.2917, 3.9848,
        1.2917, 1.2917, 1.2917, 2.4093, 2.0132, 0.9108, 1.2597, 1.2017, 1.2917,
        1.2917, 1.2917, 2.2239, 1.2917, 0.7512, 1.2917, 1.2170, 2.2608, 1.2917,
        2.8851, 1.2917, 3.3500, 1.2917, 1.2917, 1.2917, 2.4532, 1.1275, 1.2917,
        2.6637, 2.1153, 1.2917, 0.9961, 1.0828, 1.7866, 1.1821, 1.2917, 0.9425]),
indices=tensor([2, 0, 1, 2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 1, 1, 1, 0, 0, 0, 2, 0, 1, 0,
        1, 2, 0, 2, 0, 2, 0, 0, 0, 2, 1, 0, 2, 2, 0, 1, 1, 2, 1, 0, 1]))


In [ ]:

# Evaluate the model:
total_correct = 0
total_data = 0
with torch.no_grad():  # Disable gradient calculation for testing
    for images, labels in test_dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class label
        print(predicted)
        print(torch.max(labels,1)[1])
        total_correct += (predicted == torch.max(labels,1)[1]).sum().item()  # Count correct predictions
        total_data += labels.size(0)

accuracy = total_correct / total_data
print('Accuracy of the network on the test images: %d %%' % (100 * accuracy))


tensor([2, 0, 1, 2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 1, 1, 1, 0, 0, 0, 2, 0, 1, 0,
        1, 2, 0, 2, 0, 2, 0, 0, 0, 2, 1, 0, 2, 2, 0, 1, 1, 2, 1, 0, 1])
tensor([2, 0, 1, 2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 1, 1, 1, 0, 0, 0, 2, 0, 1, 0,
        1, 2, 0, 2, 0, 2, 0, 0, 0, 2, 1, 0, 2, 2, 0, 1, 1, 2, 1, 0, 1])
Accuracy of the network on the test images: 100 %
